In [1]:
import glob
import pandas as pd
import numpy as np
import json

In [2]:
annotations=glob.glob('data/OPP-115/annotations/*')

In [3]:
annotations_list=[]
for file in annotations:
    annotations_list.append(pd.read_csv(file, header=None))

In [4]:
annot=pd.concat(annotations_list, ignore_index=True)

In [5]:
annot.columns = ['annotation_ID', 'batch_ID', 'annotator_ID', 'policy_ID', 'segment_ID','category_name',
            'attribute_value_pairs','date','policy_URL']

In [6]:
annot['dict'] = annot.attribute_value_pairs.apply(lambda x: json.loads(x))

In [7]:
annot.head()

,annotation_ID,batch_ID,annotator_ID,policy_ID,segment_ID,category_name,attribute_value_pairs,date,policy_URL,dict
0,13160,test_category_labeling_highlight_fordham_aaaa,121,3828,0,Other,"{""Other Type"": {""endIndexInSegment"": 575, ""sta...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Other Type': {'endIndexInSegment': 575, 'sta..."
1,13161,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
2,13162,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
3,13163,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
4,13164,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."


In [8]:
for x in annot['dict']:
    for key, value in x.items():
        value.pop('endIndexInSegment', None)
        value.pop('startIndexInSegment', None)

In [9]:
text=[]
category=[]
subcat=[]
label=[]
counter=0
for i,x in enumerate(annot['dict']):
    for key, value in x.items():
        subcat.append(key)
        if value.get('selectedText')==None:
            text.append('noSelectedText')
        else:
            text.append(value.get('selectedText'))  
        category.append(annot['category_name'][i])
        for k, v in value.items():
            if k=='value':
                label.append(v)

In [10]:
d=list(zip(text, category, subcat, label))

In [11]:
data=pd.DataFrame(d, columns=['text', 'category', 'subcategory', 'label'])

In [12]:
data.head()

,text,category,subcategory,label
0,"Effective Date: May 7, 2015 Kraft Site Privacy...",Other,Other Type,Introductory/Generic
1,noSelectedText,First Party Collection/Use,Collection Mode,not-selected
2,collec,First Party Collection/Use,Choice Scope,Collection
3,register on our website or participate in our ...,First Party Collection/Use,Action First-Party,Collect on website
4,"personally-identifiable information, such as",First Party Collection/Use,Personal Information Type,Generic personal information


In [13]:
import string, re
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct
data['text'] = data['text'].apply(lambda x : remove_punct(x.lower()))

data = data[data['text']!='null']
data = data[data['text']!='noselectedtext']
data = data[data['text']!='']
data = data[data['text']!=' ']
data.reset_index(inplace=True, drop=True)
ds = data.drop_duplicates()

In [14]:
ds.head()

,text,category,subcategory,label
0,effective date may 7 2015 kraft site privacy n...,Other,Other Type,Introductory/Generic
1,collec,First Party Collection/Use,Choice Scope,Collection
2,register on our website or participate in our ...,First Party Collection/Use,Action First-Party,Collect on website
3,personallyidentifiable information such as,First Party Collection/Use,Personal Information Type,Generic personal information
4,if you choose to register on our website or pa...,First Party Collection/Use,Choice Type,Opt-in


In [15]:
ds['tokenized'] = ds['text'].apply(lambda x : re.split(' ', x))

/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
from gensim.models import Word2Vec
import logging
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [17]:
w2v = Word2Vec.load('word2vec.model')

2019-09-19 21:59:46,671 : INFO : loading Word2Vec object from word2vec.model
/miniconda3/envs/tensorflow/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-09-19 21:59:46,714 : INFO : loading wv recursively from word2vec.model.wv.* with mmap=None
2019-09-19 21:59:46,715 : INFO : setting ignored attribute vectors_norm to None
2019-09-19 21:59:46,717 : INFO : loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
2019-09-19 21:59:46,719 : INFO : loading trainables recursively from word2vec.model.trainables.* with mmap=None
2019-09-19 21:59:46,721 : INFO : setting ignored attribute cum_table to None
2019-09-19 21:59:46,722 : INFO : loaded word2vec.model


In [18]:
ds.reset_index(inplace=True, drop=True)

In [19]:
word_vectors = w2v.wv

In [20]:
word2id = {k: v.index for k, v in word_vectors.vocab.items()}

In [21]:
def add_new_word(new_word, new_vector, new_index, embedding_matrix, word2id):
    embedding_matrix = np.insert(embedding_matrix, [new_index], [new_vector], axis=0)
    
    word2id = {word: (index+1) if index >= new_index else index for word, index in word2id.items()}
    word2id[new_word] = new_index
    return embedding_matrix, word2id

UNK_INDEX = 0
UNK_TOKEN = 'UNK'

embedding_matrix = word_vectors.vectors
unk_vector = embedding_matrix.mean(0)
embedding_matrix, word2id = add_new_word(UNK_TOKEN, unk_vector, UNK_INDEX, embedding_matrix, word2id)

In [22]:
embedding_matrix = np.insert(embedding_matrix, len(embedding_matrix), [np.zeros((100,))], axis=0)

In [23]:
def get_int_data(token_text, word2id):
    x = []
    unk_count = 0
    for item in token_text:
        temp=[]
        x.append(temp)
        for word in item:
            if word in word2id:
                temp.append(word2id.get(word))
            else:
                temp.append(UNK_INDEX)
                unk_count += 1
    print('Data created. Percentage of unknown words: %.3f' % (unk_count))
    return np.array(x)

x=get_int_data(ds.tokenized, word2id)

Data created. Percentage of unknown words: 7006.000


In [24]:
ds['enumerated_text']=x

/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
ds.groupby(['category','subcategory']).count()

text  label  \
category                             subcategory                              
Data Retention                       Personal Information Type   188    188   
                                     Retention Period            234    234   
                                     Retention Purpose           254    254   
Data Security                        Security Measure            808    808   
Do Not Track                         Do Not Track policy          67     67   
First Party Collection/Use           Action First-Party         2733   2733   
                                     Choice Scope                528    528   
                                     Choice Type                 937    937   
                                     Collection Mode            1528   1528   
                                     Does/Does Not              1308   1308   
                                     Identifiability             636    636   
                                     Personal Information Type  3119   3119   
                                     Purpose                    4214   4214   
                                     User Type                   343    343   
International and Specific Audiences Audience Type               582    582   
Other                                Other Type                 2759   2759   
Policy Change                        Change Type                 341    341   
                                     Notification Type           403    403   
                                     User Choice                 188    188   
Third Party Sharing/Collection       Action Third Party         1889   1889   
                                     Choice Scope                342    342   
                                     Choice Type                 681    681   
                                     Does/Does Not               937    937   
                                     Identifiability             386    386   
                                     Personal Information Type  1463   1463   
                                     Purpose                    2727   2727   
                                     Third Party Entity         2094   2094   
                                     User Type                   136    136   
User Access, Edit and Deletion       Access Scope                421    421   
                                     Access Type                 514    514   
                                     User Type                   159    159   
User Choice/Control                  Choice Scope               1099   1099   
                                     Choice Type                1262   1262   
                                     Personal Information Type   492    492   
                                     Purpose                     802    802   
                                     User Type                   143    143   

                                                                tokenized  \
category                             subcategory                            
Data Retention                       Personal Information Type        188   
                                     Retention Period                 234   
                                     Retention Purpose                254   
Data Security                        Security Measure                 808   
Do Not Track                         Do Not Track policy               67   
First Party Collection/Use           Action First-Party              2733   
                                     Choice Scope                     528   
                                     Choice Type                      937   
                                     Collection Mode                 1528   
                                     Does/Does Not                   1308   
                                     Identifiability                  636   
                                     Personal Information Ty

In [26]:
ds.head()

,text,category,subcategory,label,tokenized,enumerated_text
0,effective date may 7 2015 kraft site privacy n...,Other,Other Type,Introductory/Generic,"[effective, date, may, 7, 2015, kraft, site, p...","[553, 244, 11, 2150, 839, 1332, 35, 26, 139, 2..."
1,collec,First Party Collection/Use,Choice Scope,Collection,[collec],[2151]
2,register on our website or participate in our ...,First Party Collection/Use,Action First-Party,Collect on website,"[register, on, our, website, or, participate, ...","[204, 19, 10, 52, 6, 212, 13, 10, 168, 3, 363]"
3,personallyidentifiable information such as,First Party Collection/Use,Personal Information Type,Generic personal information,"[personallyidentifiable, information, such, as]","[630, 5, 31, 23]"
4,if you choose to register on our website or pa...,First Party Collection/Use,Choice Type,Opt-in,"[if, you, choose, to, register, on, our, websi...","[32, 4, 105, 1, 204, 19, 10, 52, 6, 212, 13, 1..."


In [27]:
Data_Retention_Personal_Information_Type = ds[(ds['category']=='Data Retention') & (ds['subcategory']=='Personal Information Type')]
Data_Retention_Retention_Period = ds[(ds['category']=='Data Retention') & (ds['subcategory']=='Retention Period')]
Data_Retention_Retention_Purpose = ds[(ds['category']=='Data Retention') & (ds['subcategory']=='Retention Purpose')]

Data_Security_Security_Measure = ds[(ds['category']=='Data Security') & (ds['subcategory']=='Security Measure')]

Do_Not_Track_Do_Not_Track_Policy = ds[(ds['category']=='Do Not Track') & (ds['subcategory']=='Do Not Track policy')]

First_Party_Collection_Use_Action_First_Party = ds[(ds['category']=='First Party Collection/Use') & (ds['subcategory']=='Action First-Party')]
First_Party_Collection_Use_Choice_Scope = ds[(ds['category']=='First Party Collection/Use') & (ds['subcategory']=='Choice Scope')]
First_Party_Collection_Use_Choice_Type = ds[(ds['category']=='First Party Collection/Use') & (ds['subcategory']=='Choice Type')]
First_Party_Collection_Use_Collection_Mode = ds[(ds['category']=='First Party Collection/Use') & (ds['subcategory']=='Collection Mode')]
First_Party_Collection_Use_Does_Does_Not = ds[(ds['category']=='First Party Collection/Use') & (ds['subcategory']=='Does/Does Not')]
First_Party_Collection_Use_Identifiability = ds[(ds['category']=='First Party Collection/Use') & (ds['subcategory']=='Identifiability')]
First_Party_Collection_Use_Personal_Information_Type = ds[(ds['category']=='First Party Collection/Use') & (ds['subcategory']=='Personal Information Type')]
First_Party_Collection_Use_Purpose = ds[(ds['category']=='First Party Collection/Use') & (ds['subcategory']=='Purpose')]
First_Party_Collection_Use_User_Type = ds[(ds['category']=='First Party Collection/Use') & (ds['subcategory']=='User Type')]

International_And_Specific_Audiences_Audience_Type = ds[(ds['category']=='International and Specific Audiences') & (ds['subcategory']=='Audience Type')]

Other_Other_Type = ds[(ds['category']=='Other') & (ds['subcategory']=='Other Type')]

Policy_Change_Change_Type = ds[(ds['category']=='Policy Change') & (ds['subcategory']=='Change Type')]
Policy_Change_Notification_Type = ds[(ds['category']=='Policy Change') & (ds['subcategory']=='Notification Type')]
Policy_Change_User_Choice = ds[(ds['category']=='Policy Change') & (ds['subcategory']=='User Choice')]

Third_Party_Sharing_Collection_Action_Third_Party = ds[(ds['category']=='Third Party Sharing/Collection') & (ds['subcategory']=='Action Third Party')]
Third_Party_Sharing_Collection_Choice_Scope = ds[(ds['category']=='Third Party Sharing/Collection') & (ds['subcategory']=='Choice Scope')]
Third_Party_Sharing_Collection_Choice_Type = ds[(ds['category']=='Third Party Sharing/Collection') & (ds['subcategory']=='Choice Type')]
Third_Party_Sharing_Collection_Does_Does_Not = ds[(ds['category']=='Third Party Sharing/Collection') & (ds['subcategory']=='Does/Does Not')]
Third_Party_Sharing_Collection_Identifiability = ds[(ds['category']=='Third Party Sharing/Collection') & (ds['subcategory']=='Identifiability')]
Third_Party_Sharing_Collection_Personal_Information_Type = ds[(ds['category']=='Third Party Sharing/Collection') & (ds['subcategory']=='Personal Information Type')]
Third_Party_Sharing_Collection_Purpose = ds[(ds['category']=='Third Party Sharing/Collection') & (ds['subcategory']=='Purpose')]
Third_Party_Sharing_Collection_Third_Party_Entity = ds[(ds['category']=='Third Party Sharing/Collection') & (ds['subcategory']=='Third Party Entity')]
Third_Party_Sharing_Collection_User_Type = ds[(ds['category']=='Third Party Sharing/Collection') & (ds['subcategory']=='User Type')]

User_Access_Edit_And_Deletion_Access_Scope = ds[(ds['category']=='User Access, Edit and Deletion') & (ds['subcategory']=='Access Scope')]
User_Access_Edit_And_Deletion_Access_Type = ds[(ds['category']=='User Access, Edit and Deletion') & (ds['subcategory']=='Access Type')]
User_Access_Edit_And_Deletion_User_Type = ds[(ds['category']=='User Access, Edit and Deletion') & (ds['subcategory']=='User Type')]

User_Choice_Control_Choice_Scope = ds[(ds['category']=='User Choice/Control') & (ds['subcategory']=='Choice Scope')]
User_Choice_Control_Choice_Type = ds[(ds['category']=='User Choice/Control') & (ds['subcategory']=='Choice Type')]
User_Choice_Control_Personal_Information_Type = ds[(ds['category']=='User Choice/Control') & (ds['subcategory']=='Personal Information Type')]
User_Choice_Control_Purpose = ds[(ds['category']=='User Choice/Control') & (ds['subcategory']=='Purpose')]
User_Choice_Control_User_Type = ds[(ds['category']=='User Choice/Control') & (ds['subcategory']=='User Type')]

In [28]:
Data_Retention_Personal_Information_Type.reset_index(inplace=True, drop=True)
Data_Retention_Retention_Period.reset_index(inplace=True, drop=True)
Data_Retention_Retention_Purpose.reset_index(inplace=True, drop=True)

Data_Security_Security_Measure.reset_index(inplace=True, drop=True)

Do_Not_Track_Do_Not_Track_Policy.reset_index(inplace=True, drop=True)

First_Party_Collection_Use_Action_First_Party.reset_index(inplace=True, drop=True)
First_Party_Collection_Use_Choice_Scope.reset_index(inplace=True, drop=True)
First_Party_Collection_Use_Choice_Type.reset_index(inplace=True, drop=True)
First_Party_Collection_Use_Collection_Mode.reset_index(inplace=True, drop=True)
First_Party_Collection_Use_Does_Does_Not.reset_index(inplace=True, drop=True)
First_Party_Collection_Use_Identifiability.reset_index(inplace=True, drop=True)
First_Party_Collection_Use_Personal_Information_Type.reset_index(inplace=True, drop=True)
First_Party_Collection_Use_Purpose.reset_index(inplace=True, drop=True)
First_Party_Collection_Use_User_Type.reset_index(inplace=True, drop=True)

International_And_Specific_Audiences_Audience_Type.reset_index(inplace=True, drop=True)

Other_Other_Type.reset_index(inplace=True, drop=True)

Policy_Change_Change_Type.reset_index(inplace=True, drop=True)
Policy_Change_Notification_Type.reset_index(inplace=True, drop=True)
Policy_Change_User_Choice.reset_index(inplace=True, drop=True)

Third_Party_Sharing_Collection_Third_Party_Entity.reset_index(inplace=True, drop=True)
Third_Party_Sharing_Collection_Action_Third_Party.reset_index(inplace=True, drop=True)
Third_Party_Sharing_Collection_Choice_Scope.reset_index(inplace=True, drop=True)
Third_Party_Sharing_Collection_Choice_Type.reset_index(inplace=True, drop=True)
Third_Party_Sharing_Collection_Does_Does_Not.reset_index(inplace=True, drop=True)
Third_Party_Sharing_Collection_Identifiability.reset_index(inplace=True, drop=True)
Third_Party_Sharing_Collection_Personal_Information_Type.reset_index(inplace=True, drop=True)
Third_Party_Sharing_Collection_Purpose.reset_index(inplace=True, drop=True)
Third_Party_Sharing_Collection_User_Type.reset_index(inplace=True, drop=True)

User_Access_Edit_And_Deletion_Access_Scope.reset_index(inplace=True, drop=True)
User_Access_Edit_And_Deletion_Access_Type.reset_index(inplace=True, drop=True)
User_Access_Edit_And_Deletion_User_Type.reset_index(inplace=True, drop=True)

User_Choice_Control_Choice_Scope.reset_index(inplace=True, drop=True)
User_Choice_Control_Choice_Type.reset_index(inplace=True, drop=True)
User_Choice_Control_Personal_Information_Type.reset_index(inplace=True, drop=True)
User_Choice_Control_Purpose.reset_index(inplace=True, drop=True)
User_Choice_Control_User_Type.reset_index(inplace=True, drop=True)

In [29]:
max_length = max(ds.enumerated_text.apply(lambda x: len(x)))

In [30]:
from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder()

In [31]:
#Data_Retention_Personal_Information_Type
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix, f1_score
from sklearn import preprocessing
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation, Flatten


padded_docs = pad_sequences(Data_Retention_Personal_Information_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Data_Retention_Personal_Information_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Data_Retention_Personal_Information_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

import pickle

filename = 'data_retention_personal_information_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('data_retention_personal_information_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Using TensorFlow backend.


Train on 131 samples, validate on 57 samples
Epoch 1/10
131/131 [==============================] - 0s 4ms/step - loss: 2.7671 - acc: 0.0916 - val_loss: 2.6258 - val_acc: 0.2807
Epoch 2/10
131/131 [==============================] - 0s 1ms/step - loss: 2.3163 - acc: 0.4962 - val_loss: 2.4909 - val_acc: 0.2982
Epoch 3/10
131/131 [==============================] - 0s 813us/step - loss: 2.0080 - acc: 0.5038 - val_loss: 2.3979 - val_acc: 0.2807
Epoch 4/10
131/131 [==============================] - 0s 830us/step - loss: 1.7601 - acc: 0.5573 - val_loss: 2.2937 - val_acc: 0.3158
Epoch 5/10
131/131 [==============================] - 0s 857us/step - loss: 1.5605 - acc: 0.6641 - val_loss: 2.2080 - val_acc: 0.3333
Epoch 6/10
131/131 [==============================] - 0s 830us/step - loss: 1.3914 - acc: 0.6870 - val_loss: 2.1487 - val_acc: 0.3509
Epoch 7/10
131/131 [==============================] - 0s 807us/step - loss: 1.2625 - acc: 0.7176 - val_loss: 2.1050 - val_acc: 0.3509
Epoch 8/10
131/131 [=

In [32]:
#Data Retention-Retention Period

padded_docs = pad_sequences(Data_Retention_Retention_Period.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Data_Retention_Retention_Period.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Data_Retention_Retention_Period.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'data_retention_rentention_period_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('data_retention_rentention_period_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 163 samples, validate on 71 samples
Epoch 1/10
163/163 [==============================] - 1s 3ms/step - loss: 1.5681 - acc: 0.2945 - val_loss: 1.5744 - val_acc: 0.3521
Epoch 2/10
163/163 [==============================] - 0s 813us/step - loss: 1.0759 - acc: 0.6564 - val_loss: 1.5755 - val_acc: 0.4225
Epoch 3/10
163/163 [==============================] - 0s 683us/step - loss: 0.8179 - acc: 0.7423 - val_loss: 1.5661 - val_acc: 0.3662
Epoch 4/10
163/163 [==============================] - 0s 727us/step - loss: 0.6755 - acc: 0.7791 - val_loss: 1.6299 - val_acc: 0.3944
Epoch 5/10
163/163 [==============================] - 0s 685us/step - loss: 0.5571 - acc: 0.8282 - val_loss: 1.7580 - val_acc: 0.3803
Epoch 6/10
163/163 [==============================] - 0s 692us/step - loss: 0.4732 - acc: 0.8712 - val_loss: 1.8526 - val_acc: 0.3662
Epoch 7/10
163/163 [==============================] - 0s 709us/step - loss: 0.4041 - acc: 0.8589 - val_loss: 1.9402 - val_acc: 0.3380
Epoch 8/10
163/163 

In [33]:
#Data Retention-Retention Purpose

padded_docs = pad_sequences(Data_Retention_Retention_Purpose.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Data_Retention_Retention_Purpose.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Data_Retention_Retention_Purpose.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'data_retention_retention_purpose_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('data_retention_retention_purpose_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 177 samples, validate on 77 samples
Epoch 1/10
177/177 [==============================] - 1s 3ms/step - loss: 2.0844 - acc: 0.1751 - val_loss: 1.9212 - val_acc: 0.2468
Epoch 2/10
177/177 [==============================] - 0s 639us/step - loss: 1.5582 - acc: 0.6384 - val_loss: 1.7684 - val_acc: 0.3766
Epoch 3/10
177/177 [==============================] - 0s 625us/step - loss: 1.2367 - acc: 0.7006 - val_loss: 1.6461 - val_acc: 0.4416
Epoch 4/10
177/177 [==============================] - 0s 623us/step - loss: 1.0043 - acc: 0.7684 - val_loss: 1.5910 - val_acc: 0.4416
Epoch 5/10
177/177 [==============================] - 0s 646us/step - loss: 0.8384 - acc: 0.8079 - val_loss: 1.6264 - val_acc: 0.4805
Epoch 6/10
177/177 [==============================] - 0s 669us/step - loss: 0.7101 - acc: 0.8644 - val_loss: 1.6664 - val_acc: 0.4545
Epoch 7/10
177/177 [==============================] - 0s 683us/step - loss: 0.5924 - acc: 0.8701 - val_loss: 1.6555 - val_acc: 0.4545
Epoch 8/10
177/177 

In [34]:
#Data Security-Security Measure

padded_docs = pad_sequences(Data_Security_Security_Measure.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Data_Security_Security_Measure.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Data_Security_Security_Measure.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'data_security_security_measure_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('data_security_security_measure_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 565 samples, validate on 243 samples
Epoch 1/10
565/565 [==============================] - 1s 2ms/step - loss: 2.0194 - acc: 0.3027 - val_loss: 1.7958 - val_acc: 0.3539
Epoch 2/10
565/565 [==============================] - 0s 589us/step - loss: 1.2821 - acc: 0.5841 - val_loss: 1.6569 - val_acc: 0.4691
Epoch 3/10
565/565 [==============================] - 0s 671us/step - loss: 0.9151 - acc: 0.7540 - val_loss: 1.6478 - val_acc: 0.4938
Epoch 4/10
565/565 [==============================] - 0s 613us/step - loss: 0.6852 - acc: 0.8319 - val_loss: 1.6272 - val_acc: 0.5144
Epoch 5/10
565/565 [==============================] - 0s 674us/step - loss: 0.5179 - acc: 0.8779 - val_loss: 1.6650 - val_acc: 0.5144
Epoch 6/10
565/565 [==============================] - 0s 627us/step - loss: 0.4050 - acc: 0.8938 - val_loss: 1.6894 - val_acc: 0.5185
Epoch 7/10
565/565 [==============================] - 0s 662us/step - loss: 0.3263 - acc: 0.9062 - val_loss: 1.7402 - val_acc: 0.5103
Epoch 8/10
565/565

In [35]:
#Do Not Track-Do Not Track policy

padded_docs = pad_sequences(Do_Not_Track_Do_Not_Track_Policy.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Do_Not_Track_Do_Not_Track_Policy.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Do_Not_Track_Do_Not_Track_Policy.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'do_not_track_do_not_track_policy_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('do_not_track_do_not_track_policy_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 46 samples, validate on 21 samples
Epoch 1/10
46/46 [==============================] - 1s 13ms/step - loss: 1.5775 - acc: 0.1957 - val_loss: 0.8118 - val_acc: 0.8571
Epoch 2/10
46/46 [==============================] - 0s 1ms/step - loss: 0.6867 - acc: 0.8696 - val_loss: 0.6175 - val_acc: 0.8571
Epoch 3/10
46/46 [==============================] - 0s 1ms/step - loss: 0.4460 - acc: 0.9130 - val_loss: 0.5213 - val_acc: 0.8571
Epoch 4/10
46/46 [==============================] - 0s 1ms/step - loss: 0.3136 - acc: 0.9130 - val_loss: 0.4831 - val_acc: 0.8571
Epoch 5/10
46/46 [==============================] - 0s 893us/step - loss: 0.2399 - acc: 0.9130 - val_loss: 0.4624 - val_acc: 0.8571
Epoch 6/10
46/46 [==============================] - 0s 916us/step - loss: 0.1970 - acc: 0.9348 - val_loss: 0.4494 - val_acc: 0.8571
Epoch 7/10
46/46 [==============================] - 0s 951us/step - loss: 0.1684 - acc: 0.9348 - val_loss: 0.4386 - val_acc: 0.9048
Epoch 8/10
46/46 [=====================

In [36]:
#First Party Collection/Use-Action First-Party

padded_docs = pad_sequences(First_Party_Collection_Use_Action_First_Party.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(First_Party_Collection_Use_Action_First_Party.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, First_Party_Collection_Use_Action_First_Party.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))


filename = 'first_party_collection_use_action_first_party_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('first_party_collection_use_action_first_party_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 1913 samples, validate on 820 samples
Epoch 1/10
1913/1913 [==============================] - 2s 952us/step - loss: 1.7450 - acc: 0.4187 - val_loss: 1.5783 - val_acc: 0.4378
Epoch 2/10
1913/1913 [==============================] - 1s 649us/step - loss: 1.3403 - acc: 0.5149 - val_loss: 1.4934 - val_acc: 0.4537
Epoch 3/10
1913/1913 [==============================] - 1s 617us/step - loss: 1.1201 - acc: 0.6053 - val_loss: 1.4478 - val_acc: 0.4695
Epoch 4/10
1913/1913 [==============================] - 1s 606us/step - loss: 0.9478 - acc: 0.6733 - val_loss: 1.4572 - val_acc: 0.4841
Epoch 5/10
1913/1913 [==============================] - 1s 612us/step - loss: 0.8160 - acc: 0.7167 - val_loss: 1.4967 - val_acc: 0.4659
Epoch 6/10
1913/1913 [==============================] - 1s 603us/step - loss: 0.7131 - acc: 0.7480 - val_loss: 1.5315 - val_acc: 0.4829
Epoch 7/10
1913/1913 [==============================] - 1s 629us/step - loss: 0.6213 - acc: 0.7935 - val_loss: 1.5661 - val_acc: 0.4805
E

In [37]:
#First Party Collection/Use-Choice Scope

padded_docs = pad_sequences(First_Party_Collection_Use_Choice_Scope.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(First_Party_Collection_Use_Choice_Scope.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, First_Party_Collection_Use_Choice_Scope.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'first_party_collection_use_choice_scope_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('first_party_collection_use_choice_scope_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 369 samples, validate on 159 samples
Epoch 1/10
369/369 [==============================] - 1s 3ms/step - loss: 1.2383 - acc: 0.5122 - val_loss: 0.8956 - val_acc: 0.6730
Epoch 2/10
369/369 [==============================] - 1s 1ms/step - loss: 0.8233 - acc: 0.6558 - val_loss: 0.8293 - val_acc: 0.6667
Epoch 3/10
369/369 [==============================] - 0s 944us/step - loss: 0.6644 - acc: 0.7615 - val_loss: 0.8427 - val_acc: 0.6918
Epoch 4/10
369/369 [==============================] - 0s 651us/step - loss: 0.5396 - acc: 0.8374 - val_loss: 0.8417 - val_acc: 0.7107
Epoch 5/10
369/369 [==============================] - 0s 654us/step - loss: 0.4484 - acc: 0.8482 - val_loss: 0.8613 - val_acc: 0.7296
Epoch 6/10
369/369 [==============================] - 0s 680us/step - loss: 0.3837 - acc: 0.8591 - val_loss: 0.9076 - val_acc: 0.7107
Epoch 7/10
369/369 [==============================] - 0s 680us/step - loss: 0.3255 - acc: 0.8916 - val_loss: 0.9345 - val_acc: 0.7044
Epoch 8/10
369/369 [

In [38]:
#First_Party_Collection_Use_Choice_Type

padded_docs = pad_sequences(First_Party_Collection_Use_Choice_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(First_Party_Collection_Use_Choice_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, First_Party_Collection_Use_Choice_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'first_party_collection_use_choice_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('first_party_collection_use_choice_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 655 samples, validate on 282 samples
Epoch 1/10
655/655 [==============================] - 1s 2ms/step - loss: 1.9262 - acc: 0.3435 - val_loss: 1.6911 - val_acc: 0.4326
Epoch 2/10
655/655 [==============================] - 0s 641us/step - loss: 1.2666 - acc: 0.5634 - val_loss: 1.5663 - val_acc: 0.4787
Epoch 3/10
655/655 [==============================] - 0s 635us/step - loss: 1.0017 - acc: 0.6870 - val_loss: 1.4669 - val_acc: 0.5284
Epoch 4/10
655/655 [==============================] - 0s 677us/step - loss: 0.8154 - acc: 0.7603 - val_loss: 1.4394 - val_acc: 0.5284
Epoch 5/10
655/655 [==============================] - 0s 608us/step - loss: 0.6673 - acc: 0.8015 - val_loss: 1.4397 - val_acc: 0.5177
Epoch 6/10
655/655 [==============================] - 0s 635us/step - loss: 0.5621 - acc: 0.8321 - val_loss: 1.4553 - val_acc: 0.5142
Epoch 7/10
655/655 [==============================] - 0s 699us/step - loss: 0.4763 - acc: 0.8504 - val_loss: 1.4727 - val_acc: 0.5319
Epoch 8/10
655/655

In [39]:
#First Party Collection/Use-Collection Mode

padded_docs = pad_sequences(First_Party_Collection_Use_Collection_Mode.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(First_Party_Collection_Use_Collection_Mode.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, First_Party_Collection_Use_Collection_Mode.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'first_party_collection_collection_mode_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('first_party_collection_collection_mode_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 1069 samples, validate on 459 samples
Epoch 1/10
1069/1069 [==============================] - 1s 1ms/step - loss: 0.7702 - acc: 0.6848 - val_loss: 0.5659 - val_acc: 0.7865
Epoch 2/10
1069/1069 [==============================] - 1s 624us/step - loss: 0.4557 - acc: 0.8260 - val_loss: 0.5255 - val_acc: 0.8235
Epoch 3/10
1069/1069 [==============================] - 1s 614us/step - loss: 0.3515 - acc: 0.8765 - val_loss: 0.5229 - val_acc: 0.8279
Epoch 4/10
1069/1069 [==============================] - 1s 621us/step - loss: 0.2890 - acc: 0.8943 - val_loss: 0.5432 - val_acc: 0.8214
Epoch 5/10
1069/1069 [==============================] - 1s 630us/step - loss: 0.2440 - acc: 0.9130 - val_loss: 0.5616 - val_acc: 0.8322
Epoch 6/10
1069/1069 [==============================] - 1s 623us/step - loss: 0.2105 - acc: 0.9308 - val_loss: 0.5972 - val_acc: 0.8170
Epoch 7/10
1069/1069 [==============================] - 1s 609us/step - loss: 0.1803 - acc: 0.9336 - val_loss: 0.6469 - val_acc: 0.8039
Epo

In [40]:
#First Party Collection/Use-Does/Does Not

padded_docs = pad_sequences(First_Party_Collection_Use_Does_Does_Not.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(First_Party_Collection_Use_Does_Does_Not.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, First_Party_Collection_Use_Does_Does_Not.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'first_party_collection_use_does_does_not_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('first_party_collection_use_does_does_not_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 915 samples, validate on 393 samples
Epoch 1/10
915/915 [==============================] - 1s 1ms/step - loss: 0.3279 - acc: 0.8831 - val_loss: 0.2354 - val_acc: 0.9160
Epoch 2/10
915/915 [==============================] - 1s 639us/step - loss: 0.1381 - acc: 0.9563 - val_loss: 0.1847 - val_acc: 0.9491
Epoch 3/10
915/915 [==============================] - 1s 619us/step - loss: 0.0776 - acc: 0.9738 - val_loss: 0.1707 - val_acc: 0.9364
Epoch 4/10
915/915 [==============================] - 1s 614us/step - loss: 0.0623 - acc: 0.9792 - val_loss: 0.1938 - val_acc: 0.9440
Epoch 5/10
915/915 [==============================] - 1s 641us/step - loss: 0.0488 - acc: 0.9869 - val_loss: 0.1923 - val_acc: 0.9389
Epoch 6/10
915/915 [==============================] - 1s 617us/step - loss: 0.0435 - acc: 0.9858 - val_loss: 0.2058 - val_acc: 0.9440
Epoch 7/10
915/915 [==============================] - 1s 647us/step - loss: 0.0339 - acc: 0.9902 - val_loss: 0.2103 - val_acc: 0.9466
Epoch 8/10
915/915

In [41]:
#First Party Collection/Use-Identifiability

padded_docs = pad_sequences(First_Party_Collection_Use_Identifiability.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(First_Party_Collection_Use_Identifiability.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, First_Party_Collection_Use_Identifiability.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'first_party_collection_use_identifiability_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('first_party_collection_use_identifiability_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 445 samples, validate on 191 samples
Epoch 1/10
445/445 [==============================] - 1s 3ms/step - loss: 1.2515 - acc: 0.4022 - val_loss: 1.1483 - val_acc: 0.5812
Epoch 2/10
445/445 [==============================] - 0s 623us/step - loss: 0.8783 - acc: 0.6787 - val_loss: 1.0599 - val_acc: 0.6073
Epoch 3/10
445/445 [==============================] - 0s 720us/step - loss: 0.7039 - acc: 0.7596 - val_loss: 1.0262 - val_acc: 0.6126
Epoch 4/10
445/445 [==============================] - 0s 669us/step - loss: 0.5959 - acc: 0.7888 - val_loss: 1.0512 - val_acc: 0.6335
Epoch 5/10
445/445 [==============================] - 0s 650us/step - loss: 0.5210 - acc: 0.8112 - val_loss: 1.0652 - val_acc: 0.6440
Epoch 6/10
445/445 [==============================] - 0s 652us/step - loss: 0.4617 - acc: 0.8337 - val_loss: 1.0882 - val_acc: 0.6649
Epoch 7/10
445/445 [==============================] - 0s 659us/step - loss: 0.4197 - acc: 0.8607 - val_loss: 1.1423 - val_acc: 0.6492
Epoch 8/10
445/445

In [42]:
#First Party Collection/Use-Personal Information Type

padded_docs = pad_sequences(First_Party_Collection_Use_Personal_Information_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(First_Party_Collection_Use_Personal_Information_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, First_Party_Collection_Use_Personal_Information_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'first_party_collection_use_personal_information_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('first_party_collection_use_personal_information_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 2183 samples, validate on 936 samples
Epoch 1/10
2183/2183 [==============================] - 2s 1ms/step - loss: 2.1265 - acc: 0.3940 - val_loss: 1.7757 - val_acc: 0.4872
Epoch 2/10
2183/2183 [==============================] - 1s 601us/step - loss: 1.3693 - acc: 0.6207 - val_loss: 1.5067 - val_acc: 0.5556
Epoch 3/10
2183/2183 [==============================] - 1s 613us/step - loss: 1.0579 - acc: 0.7155 - val_loss: 1.4338 - val_acc: 0.5812
Epoch 4/10
2183/2183 [==============================] - 1s 604us/step - loss: 0.8737 - acc: 0.7696 - val_loss: 1.3964 - val_acc: 0.5897
Epoch 5/10
2183/2183 [==============================] - 2s 835us/step - loss: 0.7461 - acc: 0.8131 - val_loss: 1.3866 - val_acc: 0.5876
Epoch 6/10
2183/2183 [==============================] - 2s 1ms/step - loss: 0.6543 - acc: 0.8291 - val_loss: 1.3991 - val_acc: 0.5940
Epoch 7/10
2183/2183 [==============================] - 3s 1ms/step - loss: 0.5777 - acc: 0.8484 - val_loss: 1.3947 - val_acc: 0.5919
Epoch 8

In [43]:
#First Party Collection/Use-Purpose

padded_docs = pad_sequences(First_Party_Collection_Use_Purpose.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(First_Party_Collection_Use_Purpose.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, First_Party_Collection_Use_Purpose.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'first_party_collection_use_purpose_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('first_party_collection_use_purpose.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 2949 samples, validate on 1265 samples
Epoch 1/10
2949/2949 [==============================] - 3s 1ms/step - loss: 1.8390 - acc: 0.3645 - val_loss: 1.6069 - val_acc: 0.4379
Epoch 2/10
2949/2949 [==============================] - 2s 620us/step - loss: 1.2093 - acc: 0.6260 - val_loss: 1.4659 - val_acc: 0.5036
Epoch 3/10
2949/2949 [==============================] - 2s 607us/step - loss: 0.9417 - acc: 0.7043 - val_loss: 1.4685 - val_acc: 0.5059
Epoch 4/10
2949/2949 [==============================] - 2s 627us/step - loss: 0.7696 - acc: 0.7586 - val_loss: 1.4934 - val_acc: 0.5154
Epoch 5/10
2949/2949 [==============================] - 2s 626us/step - loss: 0.6386 - acc: 0.8159 - val_loss: 1.5416 - val_acc: 0.4980
Epoch 6/10
2949/2949 [==============================] - 2s 632us/step - loss: 0.5399 - acc: 0.8366 - val_loss: 1.5799 - val_acc: 0.5091
Epoch 7/10
2949/2949 [==============================] - 2s 622us/step - loss: 0.4612 - acc: 0.8623 - val_loss: 1.6546 - val_acc: 0.5004
Ep

In [44]:
#First Party Collection/Use-User Type

padded_docs = pad_sequences(First_Party_Collection_Use_User_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(First_Party_Collection_Use_User_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, First_Party_Collection_Use_User_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'first_party_collection_use_user_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('first_party_collection_use_user_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 240 samples, validate on 103 samples
Epoch 1/10
240/240 [==============================] - 1s 5ms/step - loss: 1.3000 - acc: 0.4708 - val_loss: 1.1053 - val_acc: 0.5631
Epoch 2/10
240/240 [==============================] - 0s 658us/step - loss: 0.9551 - acc: 0.5917 - val_loss: 1.0049 - val_acc: 0.6117
Epoch 3/10
240/240 [==============================] - 0s 789us/step - loss: 0.7379 - acc: 0.7583 - val_loss: 1.0158 - val_acc: 0.6117
Epoch 4/10
240/240 [==============================] - 0s 770us/step - loss: 0.6446 - acc: 0.7917 - val_loss: 1.0119 - val_acc: 0.6311
Epoch 5/10
240/240 [==============================] - 0s 797us/step - loss: 0.5493 - acc: 0.8125 - val_loss: 1.0418 - val_acc: 0.6408
Epoch 6/10
240/240 [==============================] - 0s 780us/step - loss: 0.4904 - acc: 0.8208 - val_loss: 1.0747 - val_acc: 0.6408
Epoch 7/10
240/240 [==============================] - 0s 794us/step - loss: 0.4312 - acc: 0.8625 - val_loss: 1.1045 - val_acc: 0.6117
Epoch 8/10
240/240

In [45]:
#International and Specific Audiences-Audience Type

padded_docs = pad_sequences(International_And_Specific_Audiences_Audience_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(International_And_Specific_Audiences_Audience_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, International_And_Specific_Audiences_Audience_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'international_specific_audiences_audience_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('international_specific_audiences_audience_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 407 samples, validate on 175 samples
Epoch 1/10
407/407 [==============================] - 1s 3ms/step - loss: 1.2669 - acc: 0.4816 - val_loss: 0.9065 - val_acc: 0.6914
Epoch 2/10
407/407 [==============================] - 0s 693us/step - loss: 0.5599 - acc: 0.8280 - val_loss: 0.8256 - val_acc: 0.7143
Epoch 3/10
407/407 [==============================] - 0s 658us/step - loss: 0.3919 - acc: 0.8821 - val_loss: 0.7980 - val_acc: 0.7314
Epoch 4/10
407/407 [==============================] - 0s 652us/step - loss: 0.2980 - acc: 0.9091 - val_loss: 0.7933 - val_acc: 0.7486
Epoch 5/10
407/407 [==============================] - 0s 710us/step - loss: 0.2348 - acc: 0.9165 - val_loss: 0.8032 - val_acc: 0.7486
Epoch 6/10
407/407 [==============================] - 0s 699us/step - loss: 0.2010 - acc: 0.9312 - val_loss: 0.8307 - val_acc: 0.7600
Epoch 7/10
407/407 [==============================] - 0s 685us/step - loss: 0.1782 - acc: 0.9459 - val_loss: 0.8602 - val_acc: 0.7429
Epoch 8/10
407/407

In [46]:
#Other-Other Type

padded_docs = pad_sequences(Other_Other_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Other_Other_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Other_Other_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'other_other_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('other_other_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 1931 samples, validate on 828 samples
Epoch 1/10
1931/1931 [==============================] - 3s 1ms/step - loss: 1.1719 - acc: 0.4619 - val_loss: 1.0627 - val_acc: 0.5181
Epoch 2/10
1931/1931 [==============================] - 2s 1ms/step - loss: 0.7032 - acc: 0.7276 - val_loss: 1.1582 - val_acc: 0.5169
Epoch 3/10
1931/1931 [==============================] - 1s 651us/step - loss: 0.5079 - acc: 0.8058 - val_loss: 1.2718 - val_acc: 0.5145
Epoch 4/10
1931/1931 [==============================] - 1s 636us/step - loss: 0.3776 - acc: 0.8483 - val_loss: 1.4151 - val_acc: 0.5000
Epoch 5/10
1931/1931 [==============================] - 1s 637us/step - loss: 0.3108 - acc: 0.8690 - val_loss: 1.5159 - val_acc: 0.5097
Epoch 6/10
1931/1931 [==============================] - 1s 650us/step - loss: 0.2664 - acc: 0.8757 - val_loss: 1.6457 - val_acc: 0.4952
Epoch 7/10
1931/1931 [==============================] - 1s 651us/step - loss: 0.2441 - acc: 0.8804 - val_loss: 1.7102 - val_acc: 0.5024
Epoch

In [47]:
#Policy Change-Change Type

padded_docs = pad_sequences(Policy_Change_Change_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Policy_Change_Change_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Policy_Change_Change_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'policy_change_change_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('policy_change_change_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 238 samples, validate on 103 samples
Epoch 1/10
238/238 [==============================] - 1s 5ms/step - loss: 1.3865 - acc: 0.4412 - val_loss: 1.2409 - val_acc: 0.5825
Epoch 2/10
238/238 [==============================] - 0s 718us/step - loss: 0.8489 - acc: 0.7479 - val_loss: 1.2775 - val_acc: 0.5922
Epoch 3/10
238/238 [==============================] - 0s 797us/step - loss: 0.6908 - acc: 0.7731 - val_loss: 1.3344 - val_acc: 0.5631
Epoch 4/10
238/238 [==============================] - 0s 702us/step - loss: 0.5685 - acc: 0.7899 - val_loss: 1.3316 - val_acc: 0.6311
Epoch 5/10
238/238 [==============================] - 0s 730us/step - loss: 0.4868 - acc: 0.8193 - val_loss: 1.3552 - val_acc: 0.5825
Epoch 6/10
238/238 [==============================] - 0s 709us/step - loss: 0.4300 - acc: 0.8529 - val_loss: 1.3964 - val_acc: 0.5728
Epoch 7/10
238/238 [==============================] - 0s 727us/step - loss: 0.3917 - acc: 0.8613 - val_loss: 1.4500 - val_acc: 0.5825
Epoch 8/10
238/238

In [48]:
#Policy Change-Notification Type

padded_docs = pad_sequences(Policy_Change_Notification_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Policy_Change_Notification_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Policy_Change_Notification_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'policy_change_notification_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('policy_change_notification_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 282 samples, validate on 121 samples
Epoch 1/10
282/282 [==============================] - 1s 5ms/step - loss: 1.6957 - acc: 0.3191 - val_loss: 1.4651 - val_acc: 0.6033
Epoch 2/10
282/282 [==============================] - 0s 646us/step - loss: 1.2061 - acc: 0.6560 - val_loss: 1.2924 - val_acc: 0.6033
Epoch 3/10
282/282 [==============================] - 0s 655us/step - loss: 0.9022 - acc: 0.7376 - val_loss: 1.2435 - val_acc: 0.6116
Epoch 4/10
282/282 [==============================] - 0s 605us/step - loss: 0.7223 - acc: 0.8227 - val_loss: 1.2354 - val_acc: 0.6364
Epoch 5/10
282/282 [==============================] - 0s 653us/step - loss: 0.5849 - acc: 0.8546 - val_loss: 1.2277 - val_acc: 0.6116
Epoch 6/10
282/282 [==============================] - 0s 620us/step - loss: 0.4892 - acc: 0.8759 - val_loss: 1.2126 - val_acc: 0.6198
Epoch 7/10
282/282 [==============================] - 0s 679us/step - loss: 0.4091 - acc: 0.9043 - val_loss: 1.2293 - val_acc: 0.6281
Epoch 8/10
282/282

In [49]:
#Policy Change-User Choice

padded_docs = pad_sequences(Policy_Change_User_Choice.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Policy_Change_User_Choice.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Policy_Change_User_Choice.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'policy_change_user_choice_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('policy_change_user_choice_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 131 samples, validate on 57 samples
Epoch 1/10
131/131 [==============================] - 1s 10ms/step - loss: 1.7351 - acc: 0.2901 - val_loss: 1.5575 - val_acc: 0.4912
Epoch 2/10
131/131 [==============================] - 0s 783us/step - loss: 1.0692 - acc: 0.6641 - val_loss: 1.4839 - val_acc: 0.4737
Epoch 3/10
131/131 [==============================] - 0s 798us/step - loss: 0.7948 - acc: 0.7176 - val_loss: 1.4614 - val_acc: 0.4386
Epoch 4/10
131/131 [==============================] - 0s 912us/step - loss: 0.6320 - acc: 0.8092 - val_loss: 1.4794 - val_acc: 0.4561
Epoch 5/10
131/131 [==============================] - 0s 1ms/step - loss: 0.5173 - acc: 0.8397 - val_loss: 1.5139 - val_acc: 0.4561
Epoch 6/10
131/131 [==============================] - 0s 876us/step - loss: 0.4460 - acc: 0.8779 - val_loss: 1.5664 - val_acc: 0.4561
Epoch 7/10
131/131 [==============================] - 0s 839us/step - loss: 0.3954 - acc: 0.8931 - val_loss: 1.6499 - val_acc: 0.4561
Epoch 8/10
131/131 [

In [50]:
#Third Party Sharing/Collection-Action Third Party

padded_docs = pad_sequences(Third_Party_Sharing_Collection_Action_Third_Party.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Third_Party_Sharing_Collection_Action_Third_Party.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Third_Party_Sharing_Collection_Action_Third_Party.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'third_party_sharing_collection_action_third_party_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('third_party_sharing_collection_action_third_party_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 1322 samples, validate on 567 samples
Epoch 1/10
1322/1322 [==============================] - 3s 2ms/step - loss: 1.3382 - acc: 0.5439 - val_loss: 1.1322 - val_acc: 0.6102
Epoch 2/10
1322/1322 [==============================] - 1s 618us/step - loss: 0.9356 - acc: 0.6921 - val_loss: 1.0902 - val_acc: 0.6261
Epoch 3/10
1322/1322 [==============================] - 1s 955us/step - loss: 0.7629 - acc: 0.7549 - val_loss: 1.0873 - val_acc: 0.6243
Epoch 4/10
1322/1322 [==============================] - 1s 746us/step - loss: 0.6418 - acc: 0.7920 - val_loss: 1.1227 - val_acc: 0.6243
Epoch 5/10
1322/1322 [==============================] - 1s 721us/step - loss: 0.5513 - acc: 0.8238 - val_loss: 1.1559 - val_acc: 0.6138
Epoch 6/10
1322/1322 [==============================] - 1s 1ms/step - loss: 0.4831 - acc: 0.8442 - val_loss: 1.1934 - val_acc: 0.6190
Epoch 7/10
1322/1322 [==============================] - 1s 695us/step - loss: 0.4352 - acc: 0.8676 - val_loss: 1.2659 - val_acc: 0.6049
Epoch

In [51]:
#Third Party Sharing/Collection-Choice Scope

padded_docs = pad_sequences(Third_Party_Sharing_Collection_Choice_Scope.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Third_Party_Sharing_Collection_Choice_Scope.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Third_Party_Sharing_Collection_Choice_Scope.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'third_party_sharing_collection_choice_scope_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('third_party_sharing_collection_choice_scope_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 239 samples, validate on 103 samples
Epoch 1/10
239/239 [==============================] - 4s 19ms/step - loss: 1.3608 - acc: 0.3264 - val_loss: 1.2603 - val_acc: 0.4175
Epoch 2/10
239/239 [==============================] - 0s 1ms/step - loss: 0.9519 - acc: 0.7197 - val_loss: 1.2342 - val_acc: 0.4660
Epoch 3/10
239/239 [==============================] - 0s 1ms/step - loss: 0.7450 - acc: 0.7364 - val_loss: 1.2600 - val_acc: 0.4369
Epoch 4/10
239/239 [==============================] - 0s 1ms/step - loss: 0.6011 - acc: 0.8285 - val_loss: 1.3552 - val_acc: 0.4175
Epoch 5/10
239/239 [==============================] - 0s 773us/step - loss: 0.5105 - acc: 0.8536 - val_loss: 1.4609 - val_acc: 0.3981
Epoch 6/10
239/239 [==============================] - 0s 732us/step - loss: 0.4302 - acc: 0.8787 - val_loss: 1.5199 - val_acc: 0.4078
Epoch 7/10
239/239 [==============================] - 0s 907us/step - loss: 0.3797 - acc: 0.8661 - val_loss: 1.5424 - val_acc: 0.4175
Epoch 8/10
239/239 [===

In [52]:
#Third Party Sharing/Collection-Choice Type

padded_docs = pad_sequences(Third_Party_Sharing_Collection_Choice_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Third_Party_Sharing_Collection_Choice_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Third_Party_Sharing_Collection_Choice_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'third_party_sharing_collection_choice_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('third_party_sharing_collection_choice_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 476 samples, validate on 205 samples
Epoch 1/10
476/476 [==============================] - 2s 4ms/step - loss: 2.0690 - acc: 0.2773 - val_loss: 1.7241 - val_acc: 0.4829
Epoch 2/10
476/476 [==============================] - 1s 1ms/step - loss: 1.3797 - acc: 0.5567 - val_loss: 1.5661 - val_acc: 0.5220
Epoch 3/10
476/476 [==============================] - 0s 732us/step - loss: 1.0608 - acc: 0.7038 - val_loss: 1.4749 - val_acc: 0.5171
Epoch 4/10
476/476 [==============================] - 0s 641us/step - loss: 0.8639 - acc: 0.7500 - val_loss: 1.5445 - val_acc: 0.4780
Epoch 5/10
476/476 [==============================] - 0s 719us/step - loss: 0.7074 - acc: 0.8193 - val_loss: 1.4935 - val_acc: 0.5024
Epoch 6/10
476/476 [==============================] - 0s 722us/step - loss: 0.6041 - acc: 0.8298 - val_loss: 1.5604 - val_acc: 0.4927
Epoch 7/10
476/476 [==============================] - 0s 710us/step - loss: 0.5256 - acc: 0.8634 - val_loss: 1.5881 - val_acc: 0.4927
Epoch 8/10
476/476 [

In [53]:
#Third Party Sharing/Collection-Does/Does Not

padded_docs = pad_sequences(Third_Party_Sharing_Collection_Does_Does_Not.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Third_Party_Sharing_Collection_Does_Does_Not.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Third_Party_Sharing_Collection_Does_Does_Not.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'third_party_sharing_collection_does_does_not_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('third_party_sharing_collection_does_does_not_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 655 samples, validate on 282 samples
Epoch 1/10
655/655 [==============================] - 2s 3ms/step - loss: 0.4900 - acc: 0.7267 - val_loss: 0.4037 - val_acc: 0.8369
Epoch 2/10
655/655 [==============================] - 0s 680us/step - loss: 0.2745 - acc: 0.8931 - val_loss: 0.3047 - val_acc: 0.9149
Epoch 3/10
655/655 [==============================] - 0s 653us/step - loss: 0.1596 - acc: 0.9496 - val_loss: 0.2700 - val_acc: 0.9184
Epoch 4/10
655/655 [==============================] - 0s 666us/step - loss: 0.1088 - acc: 0.9588 - val_loss: 0.2602 - val_acc: 0.9326
Epoch 5/10
655/655 [==============================] - 0s 749us/step - loss: 0.0825 - acc: 0.9695 - val_loss: 0.2689 - val_acc: 0.9326
Epoch 6/10
655/655 [==============================] - 0s 707us/step - loss: 0.0674 - acc: 0.9817 - val_loss: 0.2802 - val_acc: 0.9362
Epoch 7/10
655/655 [==============================] - 0s 632us/step - loss: 0.0516 - acc: 0.9817 - val_loss: 0.2965 - val_acc: 0.9362
Epoch 8/10
655/655

In [54]:
#Third Party Sharing/Collection-Identifiability

padded_docs = pad_sequences(Third_Party_Sharing_Collection_Identifiability.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Third_Party_Sharing_Collection_Identifiability.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Third_Party_Sharing_Collection_Identifiability.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'third_party_sharing_collection_identifiability_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('third_party_sharing_collection_identifiability_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 270 samples, validate on 116 samples
Epoch 1/10
270/270 [==============================] - 2s 6ms/step - loss: 1.3163 - acc: 0.4148 - val_loss: 1.2671 - val_acc: 0.4655
Epoch 2/10
270/270 [==============================] - 0s 699us/step - loss: 0.9941 - acc: 0.6481 - val_loss: 1.2475 - val_acc: 0.4483
Epoch 3/10
270/270 [==============================] - 0s 750us/step - loss: 0.8218 - acc: 0.7259 - val_loss: 1.1812 - val_acc: 0.5086
Epoch 4/10
270/270 [==============================] - 0s 658us/step - loss: 0.7010 - acc: 0.7926 - val_loss: 1.1606 - val_acc: 0.5172
Epoch 5/10
270/270 [==============================] - 0s 882us/step - loss: 0.6130 - acc: 0.8074 - val_loss: 1.1506 - val_acc: 0.5345
Epoch 6/10
270/270 [==============================] - 0s 747us/step - loss: 0.5518 - acc: 0.8074 - val_loss: 1.1669 - val_acc: 0.5431
Epoch 7/10
270/270 [==============================] - 0s 682us/step - loss: 0.4959 - acc: 0.8296 - val_loss: 1.1850 - val_acc: 0.5690
Epoch 8/10
270/270

In [55]:
#Third Party Sharing/Collection-Personal Information Type

padded_docs = pad_sequences(Third_Party_Sharing_Collection_Personal_Information_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Third_Party_Sharing_Collection_Personal_Information_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Third_Party_Sharing_Collection_Personal_Information_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'third_party_sharing_collection_personal_information_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('third_party_sharing_collection_personal_information_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 1024 samples, validate on 439 samples
Epoch 1/10
1024/1024 [==============================] - 2s 2ms/step - loss: 2.3549 - acc: 0.2422 - val_loss: 2.0063 - val_acc: 0.3485
Epoch 2/10
1024/1024 [==============================] - 1s 677us/step - loss: 1.6987 - acc: 0.5127 - val_loss: 1.7505 - val_acc: 0.4829
Epoch 3/10
1024/1024 [==============================] - 1s 696us/step - loss: 1.3945 - acc: 0.6064 - val_loss: 1.6034 - val_acc: 0.5057
Epoch 4/10
1024/1024 [==============================] - 1s 676us/step - loss: 1.2020 - acc: 0.6768 - val_loss: 1.5422 - val_acc: 0.5148
Epoch 5/10
1024/1024 [==============================] - 1s 690us/step - loss: 1.0563 - acc: 0.7090 - val_loss: 1.5054 - val_acc: 0.5399
Epoch 6/10
1024/1024 [==============================] - 1s 680us/step - loss: 0.9460 - acc: 0.7451 - val_loss: 1.4913 - val_acc: 0.5535
Epoch 7/10
1024/1024 [==============================] - 1s 651us/step - loss: 0.8652 - acc: 0.7578 - val_loss: 1.4923 - val_acc: 0.5330
Epo

In [56]:
#Third Party Sharing/Collection-Purpose

padded_docs = pad_sequences(Third_Party_Sharing_Collection_Purpose.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Third_Party_Sharing_Collection_Purpose.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Third_Party_Sharing_Collection_Purpose.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'third_party_sharing_collection_purpose_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('third_party_sharing_collection_purpose_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 1908 samples, validate on 819 samples
Epoch 1/10
1908/1908 [==============================] - 3s 2ms/step - loss: 1.8922 - acc: 0.3884 - val_loss: 1.5478 - val_acc: 0.5116
Epoch 2/10
1908/1908 [==============================] - 1s 636us/step - loss: 1.1365 - acc: 0.6787 - val_loss: 1.4306 - val_acc: 0.5470
Epoch 3/10
1908/1908 [==============================] - 2s 898us/step - loss: 0.8555 - acc: 0.7390 - val_loss: 1.3755 - val_acc: 0.5641
Epoch 4/10
1908/1908 [==============================] - 2s 840us/step - loss: 0.6773 - acc: 0.8077 - val_loss: 1.3637 - val_acc: 0.5714
Epoch 5/10
1908/1908 [==============================] - 1s 647us/step - loss: 0.5627 - acc: 0.8386 - val_loss: 1.3982 - val_acc: 0.5617
Epoch 6/10
1908/1908 [==============================] - 1s 658us/step - loss: 0.4977 - acc: 0.8585 - val_loss: 1.4293 - val_acc: 0.5604
Epoch 7/10
1908/1908 [==============================] - 1s 731us/step - loss: 0.4281 - acc: 0.8763 - val_loss: 1.4569 - val_acc: 0.5592
Epo

In [57]:
#Third Party Sharing/Collection-Third Party Entity

padded_docs = pad_sequences(Third_Party_Sharing_Collection_Third_Party_Entity.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Third_Party_Sharing_Collection_Third_Party_Entity.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Third_Party_Sharing_Collection_Third_Party_Entity.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'third_party_sharing_collection_third_party_entity_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('third_party_sharing_collection_third_party_entity_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 1465 samples, validate on 629 samples
Epoch 1/10
1465/1465 [==============================] - 3s 2ms/step - loss: 1.4666 - acc: 0.4198 - val_loss: 1.2717 - val_acc: 0.5183
Epoch 2/10
1465/1465 [==============================] - 1s 643us/step - loss: 1.0869 - acc: 0.5686 - val_loss: 1.2047 - val_acc: 0.5390
Epoch 3/10
1465/1465 [==============================] - 1s 710us/step - loss: 0.9096 - acc: 0.6519 - val_loss: 1.2098 - val_acc: 0.5533
Epoch 4/10
1465/1465 [==============================] - 1s 598us/step - loss: 0.7912 - acc: 0.7106 - val_loss: 1.2005 - val_acc: 0.5596
Epoch 5/10
1465/1465 [==============================] - 1s 521us/step - loss: 0.6910 - acc: 0.7440 - val_loss: 1.2254 - val_acc: 0.5580
Epoch 6/10
1465/1465 [==============================] - 1s 642us/step - loss: 0.6141 - acc: 0.7768 - val_loss: 1.2717 - val_acc: 0.5564
Epoch 7/10
1465/1465 [==============================] - 1s 665us/step - loss: 0.5552 - acc: 0.7973 - val_loss: 1.2911 - val_acc: 0.5453
Epo

In [58]:
#Third Party Sharing/Collection-User Type

padded_docs = pad_sequences(Third_Party_Sharing_Collection_User_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(Third_Party_Sharing_Collection_User_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, Third_Party_Sharing_Collection_User_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'third_party_sharing_collection_user_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('third_party_sharing_collection_user_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 95 samples, validate on 41 samples
Epoch 1/10
95/95 [==============================] - 2s 17ms/step - loss: 1.3686 - acc: 0.3684 - val_loss: 1.2939 - val_acc: 0.4146
Epoch 2/10
95/95 [==============================] - 0s 670us/step - loss: 1.1679 - acc: 0.7579 - val_loss: 1.2152 - val_acc: 0.4634
Epoch 3/10
95/95 [==============================] - 0s 641us/step - loss: 1.0316 - acc: 0.7368 - val_loss: 1.1238 - val_acc: 0.4878
Epoch 4/10
95/95 [==============================] - 0s 628us/step - loss: 0.8913 - acc: 0.7368 - val_loss: 1.0621 - val_acc: 0.5122
Epoch 5/10
95/95 [==============================] - 0s 695us/step - loss: 0.7897 - acc: 0.7158 - val_loss: 1.0390 - val_acc: 0.5122
Epoch 6/10
95/95 [==============================] - 0s 670us/step - loss: 0.7205 - acc: 0.7368 - val_loss: 1.0314 - val_acc: 0.5366
Epoch 7/10
95/95 [==============================] - 0s 676us/step - loss: 0.6557 - acc: 0.8000 - val_loss: 1.0355 - val_acc: 0.5854
Epoch 8/10
95/95 [===============

In [59]:
#User Access, Edit and Deletion-Access Scope

padded_docs = pad_sequences(User_Access_Edit_And_Deletion_Access_Scope.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(User_Access_Edit_And_Deletion_Access_Scope.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, User_Access_Edit_And_Deletion_Access_Scope.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'user_access_edit_deletion_access_scope_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('user_access_edit_deletion_access_scope_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 294 samples, validate on 127 samples
Epoch 1/10
294/294 [==============================] - 2s 6ms/step - loss: 1.6466 - acc: 0.3980 - val_loss: 1.4734 - val_acc: 0.4961
Epoch 2/10
294/294 [==============================] - 0s 567us/step - loss: 1.1996 - acc: 0.5340 - val_loss: 1.4941 - val_acc: 0.4882
Epoch 3/10
294/294 [==============================] - 0s 583us/step - loss: 0.9901 - acc: 0.6224 - val_loss: 1.4637 - val_acc: 0.5197
Epoch 4/10
294/294 [==============================] - 0s 605us/step - loss: 0.8588 - acc: 0.7109 - val_loss: 1.4696 - val_acc: 0.4961
Epoch 5/10
294/294 [==============================] - 0s 617us/step - loss: 0.7465 - acc: 0.7823 - val_loss: 1.4980 - val_acc: 0.4882
Epoch 6/10
294/294 [==============================] - 0s 718us/step - loss: 0.6538 - acc: 0.7959 - val_loss: 1.5501 - val_acc: 0.4803
Epoch 7/10
294/294 [==============================] - 0s 547us/step - loss: 0.5809 - acc: 0.8095 - val_loss: 1.5956 - val_acc: 0.4646
Epoch 8/10
294/294

In [60]:
#User Access, Edit and Deletion-Access Type

padded_docs = pad_sequences(User_Access_Edit_And_Deletion_Access_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(User_Access_Edit_And_Deletion_Access_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, User_Access_Edit_And_Deletion_Access_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'user_access_edit_deletion_access_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('user_access_edit_deletion_access_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 359 samples, validate on 155 samples
Epoch 1/10
359/359 [==============================] - 2s 6ms/step - loss: 1.9657 - acc: 0.3955 - val_loss: 1.7632 - val_acc: 0.4516
Epoch 2/10
359/359 [==============================] - 0s 603us/step - loss: 1.3366 - acc: 0.5599 - val_loss: 1.6631 - val_acc: 0.4452
Epoch 3/10
359/359 [==============================] - 0s 680us/step - loss: 1.1152 - acc: 0.6435 - val_loss: 1.6238 - val_acc: 0.4258
Epoch 4/10
359/359 [==============================] - 0s 689us/step - loss: 0.9491 - acc: 0.6936 - val_loss: 1.6168 - val_acc: 0.4516
Epoch 5/10
359/359 [==============================] - 0s 698us/step - loss: 0.8238 - acc: 0.7187 - val_loss: 1.6161 - val_acc: 0.4774
Epoch 6/10
359/359 [==============================] - 0s 664us/step - loss: 0.7294 - acc: 0.7744 - val_loss: 1.6111 - val_acc: 0.4903
Epoch 7/10
359/359 [==============================] - 0s 612us/step - loss: 0.6372 - acc: 0.8078 - val_loss: 1.6368 - val_acc: 0.4968
Epoch 8/10
359/359

In [61]:
#User Access, Edit and Deletion-User Type

padded_docs = pad_sequences(User_Access_Edit_And_Deletion_User_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(User_Access_Edit_And_Deletion_User_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, User_Access_Edit_And_Deletion_User_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'user_access_edit_deletion_user_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('user_access_edit_deletion_user_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 111 samples, validate on 48 samples
Epoch 1/10
111/111 [==============================] - 2s 17ms/step - loss: 1.3183 - acc: 0.4144 - val_loss: 0.9643 - val_acc: 0.7500
Epoch 2/10
111/111 [==============================] - 0s 1ms/step - loss: 0.8678 - acc: 0.7117 - val_loss: 0.7652 - val_acc: 0.7500
Epoch 3/10
111/111 [==============================] - 0s 885us/step - loss: 0.6641 - acc: 0.7207 - val_loss: 0.7254 - val_acc: 0.7500
Epoch 4/10
111/111 [==============================] - 0s 855us/step - loss: 0.5624 - acc: 0.7748 - val_loss: 0.6721 - val_acc: 0.7083
Epoch 5/10
111/111 [==============================] - 0s 908us/step - loss: 0.4670 - acc: 0.8198 - val_loss: 0.6743 - val_acc: 0.8125
Epoch 6/10
111/111 [==============================] - 0s 925us/step - loss: 0.4348 - acc: 0.8649 - val_loss: 0.6702 - val_acc: 0.8125
Epoch 7/10
111/111 [==============================] - 0s 903us/step - loss: 0.4031 - acc: 0.8739 - val_loss: 0.6504 - val_acc: 0.8125
Epoch 8/10
111/111 [

In [62]:
#User Choice/Control-Choice Scope

padded_docs = pad_sequences(User_Choice_Control_Choice_Scope.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(User_Choice_Control_Choice_Scope.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, User_Choice_Control_Choice_Scope.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'user_choice_control_choice_scope_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('user_choice_control_choice_scope_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 769 samples, validate on 330 samples
Epoch 1/10
769/769 [==============================] - 2s 3ms/step - loss: 1.4005 - acc: 0.4226 - val_loss: 1.2235 - val_acc: 0.5242
Epoch 2/10
769/769 [==============================] - 0s 529us/step - loss: 0.9525 - acc: 0.6437 - val_loss: 1.1521 - val_acc: 0.5273
Epoch 3/10
769/769 [==============================] - 0s 554us/step - loss: 0.7359 - acc: 0.7516 - val_loss: 1.1676 - val_acc: 0.5515
Epoch 4/10
769/769 [==============================] - 0s 535us/step - loss: 0.5919 - acc: 0.7945 - val_loss: 1.1822 - val_acc: 0.5364
Epoch 5/10
769/769 [==============================] - 0s 546us/step - loss: 0.4879 - acc: 0.8349 - val_loss: 1.2019 - val_acc: 0.5545
Epoch 6/10
769/769 [==============================] - 0s 562us/step - loss: 0.4048 - acc: 0.8674 - val_loss: 1.2453 - val_acc: 0.5545
Epoch 7/10
769/769 [==============================] - 0s 633us/step - loss: 0.3460 - acc: 0.8895 - val_loss: 1.3195 - val_acc: 0.5636
Epoch 8/10
769/769

In [63]:
#User Choice/Control-Choice Type

padded_docs = pad_sequences(User_Choice_Control_Choice_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(User_Choice_Control_Choice_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, User_Choice_Control_Choice_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'user_choice_control_choice_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('user_choice_control_choice_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 883 samples, validate on 379 samples
Epoch 1/10
883/883 [==============================] - 2s 3ms/step - loss: 2.0484 - acc: 0.2718 - val_loss: 1.8397 - val_acc: 0.4327
Epoch 2/10
883/883 [==============================] - 0s 545us/step - loss: 1.3821 - acc: 0.6251 - val_loss: 1.6038 - val_acc: 0.5172
Epoch 3/10
883/883 [==============================] - 0s 517us/step - loss: 1.0216 - acc: 0.7259 - val_loss: 1.5567 - val_acc: 0.4987
Epoch 4/10
883/883 [==============================] - 0s 516us/step - loss: 0.7763 - acc: 0.7803 - val_loss: 1.5149 - val_acc: 0.5251
Epoch 5/10
883/883 [==============================] - 1s 600us/step - loss: 0.6178 - acc: 0.8505 - val_loss: 1.5246 - val_acc: 0.5251
Epoch 6/10
883/883 [==============================] - 1s 579us/step - loss: 0.5023 - acc: 0.8698 - val_loss: 1.5731 - val_acc: 0.5119
Epoch 7/10
883/883 [==============================] - 0s 537us/step - loss: 0.4218 - acc: 0.8901 - val_loss: 1.5985 - val_acc: 0.5330
Epoch 8/10
883/883

In [64]:
#User Choice/Control-Personal Information Type

padded_docs = pad_sequences(User_Choice_Control_Personal_Information_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(User_Choice_Control_Personal_Information_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, User_Choice_Control_Personal_Information_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'user_choice_control_personal_information_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('user_choice_control_personal_information_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 344 samples, validate on 148 samples
Epoch 1/10
344/344 [==============================] - 2s 6ms/step - loss: 2.6762 - acc: 0.2442 - val_loss: 2.3635 - val_acc: 0.4932
Epoch 2/10
344/344 [==============================] - 0s 626us/step - loss: 1.9912 - acc: 0.5988 - val_loss: 1.9980 - val_acc: 0.4865
Epoch 3/10
344/344 [==============================] - 0s 692us/step - loss: 1.6148 - acc: 0.5727 - val_loss: 1.8698 - val_acc: 0.4527
Epoch 4/10
344/344 [==============================] - 0s 589us/step - loss: 1.3672 - acc: 0.6453 - val_loss: 1.7696 - val_acc: 0.5135
Epoch 5/10
344/344 [==============================] - 0s 641us/step - loss: 1.2003 - acc: 0.7209 - val_loss: 1.7266 - val_acc: 0.5135
Epoch 6/10
344/344 [==============================] - 0s 639us/step - loss: 1.0751 - acc: 0.7413 - val_loss: 1.6860 - val_acc: 0.5203
Epoch 7/10
344/344 [==============================] - 0s 604us/step - loss: 0.9776 - acc: 0.7587 - val_loss: 1.6571 - val_acc: 0.5473
Epoch 8/10
344/344

In [65]:
#User Choice/Control-Purpose

padded_docs = pad_sequences(User_Choice_Control_Purpose.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(User_Choice_Control_Purpose.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, User_Choice_Control_Purpose.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'user_choice_control_purpose_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('user_choice_control_purpose_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 561 samples, validate on 241 samples
Epoch 1/10
561/561 [==============================] - 2s 4ms/step - loss: 2.1347 - acc: 0.3369 - val_loss: 1.8039 - val_acc: 0.4149
Epoch 2/10
561/561 [==============================] - 0s 542us/step - loss: 1.3947 - acc: 0.6043 - val_loss: 1.6354 - val_acc: 0.4979
Epoch 3/10
561/561 [==============================] - 0s 550us/step - loss: 1.0863 - acc: 0.7077 - val_loss: 1.5889 - val_acc: 0.4772
Epoch 4/10
561/561 [==============================] - 0s 567us/step - loss: 0.8864 - acc: 0.7718 - val_loss: 1.5655 - val_acc: 0.4730
Epoch 5/10
561/561 [==============================] - 0s 630us/step - loss: 0.7359 - acc: 0.8146 - val_loss: 1.5584 - val_acc: 0.4813
Epoch 6/10
561/561 [==============================] - 0s 560us/step - loss: 0.6252 - acc: 0.8342 - val_loss: 1.5640 - val_acc: 0.4689
Epoch 7/10
561/561 [==============================] - 0s 613us/step - loss: 0.5412 - acc: 0.8592 - val_loss: 1.5922 - val_acc: 0.4813
Epoch 8/10
561/561

In [66]:
#User Choice/Control-User Type

padded_docs = pad_sequences(User_Choice_Control_User_Type.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

onehot=encode.fit(np.array(User_Choice_Control_User_Type.label).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(padded_docs, User_Choice_Control_User_Type.label, test_size=0.3, random_state = 0)


onehotlabels_train = encode.transform(np.array(y_train).reshape(-1,1))
onehotlabels_test = encode.transform(np.array(y_test).reshape(-1,1))


vocab_length = len(embedding_matrix)

model = Sequential()
model.add(Embedding(input_dim=vocab_length,
                   output_dim=100,
                   weights=[embedding_matrix],
                   input_length=padded_docs.shape[1]))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(onehotlabels_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(x_train, onehotlabels_train, epochs=10, batch_size=100, validation_data=(x_test, onehotlabels_test))

filename = 'user_choice_control_user_type_value_encode.sav'
pickle.dump(encode, open(filename, 'wb'))

model.save('user_choice_control_user_type_value.h5')

y_pred_train=model.predict_classes(x_train)
y_train_label=np.argmax(onehotlabels_train, axis = 1)
print(confusion_matrix(y_train_label,y_pred_train))
print(model.evaluate(x_train, onehotlabels_train))
y_pred_test=model.predict_classes(x_test)
y_test_label=np.argmax(onehotlabels_test, axis = 1)
print(confusion_matrix(y_test_label,y_pred_test))
print(model.evaluate(x_test, onehotlabels_test))

Train on 100 samples, validate on 43 samples
Epoch 1/10
100/100 [==============================] - 2s 20ms/step - loss: 1.3843 - acc: 0.2800 - val_loss: 1.2986 - val_acc: 0.6279
Epoch 2/10
100/100 [==============================] - 0s 636us/step - loss: 1.1785 - acc: 0.7700 - val_loss: 1.1735 - val_acc: 0.7442
Epoch 3/10
100/100 [==============================] - 0s 667us/step - loss: 0.9954 - acc: 0.7500 - val_loss: 1.0449 - val_acc: 0.7442
Epoch 4/10
100/100 [==============================] - 0s 622us/step - loss: 0.8326 - acc: 0.7500 - val_loss: 0.9452 - val_acc: 0.7442
Epoch 5/10
100/100 [==============================] - 0s 677us/step - loss: 0.7078 - acc: 0.7400 - val_loss: 0.8920 - val_acc: 0.7442
Epoch 6/10
100/100 [==============================] - 0s 679us/step - loss: 0.6271 - acc: 0.7400 - val_loss: 0.8727 - val_acc: 0.7442
Epoch 7/10
100/100 [==============================] - 0s 683us/step - loss: 0.5511 - acc: 0.8000 - val_loss: 0.8845 - val_acc: 0.7442
Epoch 8/10
100/100